Training a machine translation model using JW300 data and Global Voices Data from Opuss

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:

import os
source_language = "en"
target_language = "sw" 
lc = False  # If True, lowercase the data.
seed = 42
tag = "baseline" 
os.environ["src"] = source_language
os.environ["tgt"] = target_language
os.environ["tag"] = tag


In [ ]:
! pip3 install pandas
import pandas as pd

In [ ]:
####Read the train data and convert it to pandas dataframe - df
# Get JW300 data
df1=pd.read_csv('/content/drive/MyDrive/Lafand/data.csv')
df1.head()

,source_sentence,target_sentence
0,“ Look !,“ Tazama !
1,I Am Making All Things New ”,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
2,The above is a promise from God that has been ...,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo ...
3,( Revelation 21 : 5 ) What does that promise m...,"( Ufunuo 21 : 5 , NW ) Ahadi hiyo yamaanisha n..."
4,"How will it be fulfilled , and when ?","Itatimizwa jinsi gani , na wakati gani ?"


In [ ]:
len(df1)

955372

In [ ]:
####Read the train data and convert it to pandas dataframe - df
import pandas as pd
col_name_en= ['source_sentence']
col_name_sw = ['target_sentence']
global_voice_en = pd.read_csv('/content/drive/MyDrive/Lafand/GlobalVoices.en.en',  sep='\n\n', names =col_name_en)
global_voice_sw = pd.read_csv('/content/drive/MyDrive/Lafand/GlobalVoices.sw.sw', sep='\n\n', names =col_name_sw)

df2 = pd.concat([global_voice_en,global_voice_sw], axis = 1)
df = pd.concat([df1, df2])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
####print data head
print(df.tail(5))

                                         source_sentence                                    target_sentence
30777  Ugandan journalist Charles Onyango-Obbo, also ...  Mwandishi wa habari kutoka nchini Uganda Charl...
30778  Since the hunter has learned to shoot without ...  Tanguia pale mwindaji alipojifunza kulenga bil...
30779  In essence, he meant that in order to keep dig...  Akimaanisha, ili kurudisha uhuru wa mitandao m...
30780  Activists on the front lines of free speech in...  Wanaharakati walio mstari wa mbele kutetea hil...
30781  With our strength and unity, online spaces wil...  Tukileta pamoja nguvu zetu kwa umoja, hii miha...


In [ ]:
len(df)

986154

In [ ]:
# Download the test set. 
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/millicentochieng/Lafand/main/data/new_aligned/english.en
! mv english.en test.$src
! wget https://raw.githubusercontent.com/millicentochieng/Lafand/main/data/new_aligned/swahili.sw 
! mv swahili.sw test.$trg

--2021-10-25 10:47:05--  https://raw.githubusercontent.com/millicentochieng/Lafand/main/data/new_aligned/english.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 450809 (440K) [text/plain]
Saving to: ‘english.en’

english.en          100%[===================>] 440.24K  --.-KB/s    in 0.005s  

2021-10-25 10:47:05 (81.6 MB/s) - ‘english.en’ saved [450809/450809]

--2021-10-25 10:47:06--  https://raw.githubusercontent.com/millicentochieng/Lafand/main/data/new_aligned/swahili.sw
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48740

In [ ]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = 'test.en'
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1

In [ ]:
j

3725

Pre-processing and export
It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [ ]:
seed = 42  # Random seed for shuffling.
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations

df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
! pip3 install fuzzywuzzy

In [ ]:
! pip3 install python-Levenshtein

     |████████████████████████████████| 50 kB 5.6 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=e30aea87620c1cfab01c6122a1a9f83b7adca8d11a1a864b8702b4ea7d4b4ea3
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
# Use fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
from fuzzywuzzy import process
import numpy as np
import time

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(str(row['source_sentence']), list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

00:00:00.38 0.00 percent complete
00:00:21.76 0.11 percent complete
00:00:43.41 0.22 percent complete
00:01:05.38 0.34 percent complete
00:01:26.22 0.45 percent complete
00:01:48.60 0.56 percent complete
00:02:10.59 0.67 percent complete
00:02:32.28 0.79 percent complete
00:02:53.56 0.90 percent complete
00:03:15.28 1.01 percent complete
00:03:36.75 1.12 percent complete
00:03:58.95 1.24 percent complete
00:04:20.32 1.35 percent complete
00:04:41.66 1.46 percent complete
00:05:03.35 1.57 percent complete
00:05:25.38 1.69 percent complete
00:05:47.54 1.80 percent complete
00:06:08.70 1.91 percent complete
00:06:30.60 2.02 percent complete
00:06:52.04 2.14 percent complete
00:07:13.56 2.25 percent complete
00:07:35.36 2.36 percent complete


00:07:56.90 2.47 percent complete
00:08:19.02 2.59 percent complete
00:08:41.05 2.70 percent complete
00:09:02.75 2.81 percent complete
00:09:24.54 2.92 percent complete
00:09:46.05 3.04 percent complete
00:10:07.40 3.15 percent complete
00:10:29.15 3.26 percent complete
00:10:51.07 3.37 percent complete
00:11:12.64 3.48 percent complete
00:11:33.94 3.60 percent complete
00:11:55.82 3.71 percent complete
00:12:17.24 3.82 percent complete
00:12:38.74 3.93 percent complete
00:13:00.14 4.05 percent complete
00:13:21.72 4.16 percent complete
00:13:44.05 4.27 percent complete
00:14:05.61 4.38 percent complete
00:14:27.04 4.50 percent complete
00:14:48.19 4.61 percent complete
00:15:09.98 4.72 percent complete
00:15:31.73 4.83 percent complete


00:15:53.20 4.95 percent complete
00:16:15.48 5.06 percent complete
00:16:37.26 5.17 percent complete
00:16:58.34 5.28 percent complete
00:17:19.98 5.40 percent complete
00:17:41.47 5.51 percent complete
00:18:03.11 5.62 percent complete
00:18:24.91 5.73 percent complete
00:18:46.81 5.85 percent complete


00:19:08.82 5.96 percent complete
00:19:31.09 6.07 percent complete
00:19:52.44 6.18 percent complete
00:20:14.22 6.30 percent complete
00:20:35.96 6.41 percent complete
00:20:57.79 6.52 percent complete
00:21:19.37 6.63 percent complete
00:21:41.39 6.74 percent complete
00:22:02.96 6.86 percent complete
00:22:24.28 6.97 percent complete
00:22:46.01 7.08 percent complete
00:23:07.55 7.19 percent complete
00:23:29.40 7.31 percent complete
00:23:51.48 7.42 percent complete
00:24:13.39 7.53 percent complete
00:24:34.91 7.64 percent complete
00:24:57.08 7.76 percent complete
00:25:19.02 7.87 percent complete
00:25:41.03 7.98 percent complete
00:26:02.76 8.09 percent complete
00:26:23.84 8.21 percent complete
00:26:45.02 8.32 percent complete
00:27:07.42 8.43 percent complete
00:27:29.08 8.54 percent complete
00:27:50.76 8.66 percent complete
00:28:13.07 8.77 percent complete
00:28:34.97 8.88 percent complete
00:28:56.82 8.99 percent complete
00:29:18.36 9.11 percent complete
00:29:39.65 9.

00:43:24.69 13.49 percent complete
00:43:46.77 13.60 percent complete
00:44:08.16 13.71 percent complete
00:44:30.56 13.83 percent complete
00:44:52.22 13.94 percent complete
00:45:14.35 14.05 percent complete
00:45:36.18 14.16 percent complete
00:45:57.60 14.28 percent complete
00:46:19.12 14.39 percent complete
00:46:40.60 14.50 percent complete
00:47:02.52 14.61 percent complete
00:47:24.24 14.73 percent complete
00:47:45.48 14.84 percent complete
00:48:07.50 14.95 percent complete
00:48:29.10 15.06 percent complete
00:48:50.48 15.18 percent complete
00:49:11.75 15.29 percent complete
00:49:33.70 15.40 percent complete
00:49:55.48 15.51 percent complete
00:50:17.51 15.63 percent complete
00:50:39.36 15.74 percent complete
00:51:01.33 15.85 percent complete
00:51:23.87 15.96 percent complete
00:51:45.88 16.08 percent complete
00:52:07.47 16.19 percent complete
00:52:29.17 16.30 percent complete
00:52:50.47 16.41 percent complete
00:53:12.09 16.53 percent complete
00:53:33.92 16.64 pe

...

04:32:45.43 84.54 percent complete
04:33:06.78 84.65 percent complete
04:33:28.65 84.76 percent complete
04:33:50.47 84.87 percent complete
04:34:12.61 84.99 percent complete
04:34:34.09 85.10 percent complete
04:34:55.57 85.21 percent complete
04:35:17.79 85.32 percent complete
04:35:39.79 85.44 percent complete
04:36:01.56 85.55 percent complete
04:36:23.04 85.66 percent complete
04:36:44.44 85.77 percent complete
04:37:06.52 85.89 percent complete
04:37:28.45 86.00 percent complete
04:37:49.99 86.11 percent complete
04:38:11.83 86.22 percent complete
04:38:33.23 86.33 percent complete
04:38:54.90 86.45 percent complete
04:39:16.52 86.56 percent complete
04:39:38.33 86.67 percent complete
04:39:59.83 86.78 percent complete
04:40:21.59 86.90 percent complete
04:40:43.17 87.01 percent complete


04:41:05.26 87.12 percent complete
04:41:26.70 87.23 percent complete
04:41:47.84 87.35 percent complete
04:42:09.14 87.46 percent complete
04:42:30.69 87.57 percent complete
04:42:52.10 87.68 percent complete
04:43:13.86 87.80 percent complete
04:43:35.28 87.91 percent complete
04:43:57.62 88.02 percent complete
04:44:18.97 88.13 percent complete
04:44:40.16 88.25 percent complete
04:45:02.58 88.36 percent complete
04:45:24.51 88.47 percent complete
04:45:46.17 88.58 percent complete
04:46:07.92 88.70 percent complete
04:46:29.46 88.81 percent complete
04:46:50.85 88.92 percent complete
04:47:12.55 89.03 percent complete
04:47:34.86 89.15 percent complete
04:47:56.99 89.26 percent complete
04:48:18.00 89.37 percent complete
04:48:39.23 89.48 percent complete
04:49:00.77 89.59 percent complete
04:49:22.28 89.71 percent complete
04:49:43.61 89.82 percent complete
04:50:04.86 89.93 percent complete
04:50:26.56 90.04 percent complete
04:50:48.44 90.16 percent complete
04:51:10.02 90.27 pe

04:52:37.17 90.72 percent complete
04:52:58.85 90.83 percent complete
04:53:20.20 90.94 percent complete
04:53:42.17 91.06 percent complete
04:54:03.59 91.17 percent complete
04:54:25.11 91.28 percent complete
04:54:46.74 91.39 percent complete
04:55:08.71 91.51 percent complete
04:55:30.24 91.62 percent complete
04:55:51.84 91.73 percent complete
04:56:13.35 91.84 percent complete
04:56:34.87 91.96 percent complete
04:56:56.13 92.07 percent complete
04:57:18.13 92.18 percent complete
04:57:40.17 92.29 percent complete
04:58:00.93 92.41 percent complete
04:58:23.04 92.52 percent complete
04:58:44.99 92.63 percent complete
04:59:07.16 92.74 percent complete
04:59:28.31 92.85 percent complete
04:59:50.14 92.97 percent complete
05:00:12.03 93.08 percent complete
05:00:33.68 93.19 percent complete
05:00:55.19 93.30 percent complete
05:01:17.65 93.42 percent complete


05:01:39.23 93.53 percent complete
05:02:01.08 93.64 percent complete
05:02:22.79 93.75 percent complete
05:02:44.89 93.87 percent complete
05:03:06.78 93.98 percent complete
05:03:28.88 94.09 percent complete
05:03:50.50 94.20 percent complete
05:04:12.42 94.32 percent complete
05:04:33.74 94.43 percent complete
05:04:54.70 94.54 percent complete
05:05:16.71 94.65 percent complete
05:05:38.37 94.77 percent complete
05:05:59.35 94.88 percent complete
05:06:21.46 94.99 percent complete
05:06:43.66 95.10 percent complete
05:07:05.11 95.22 percent complete
05:07:26.80 95.33 percent complete
05:07:48.68 95.44 percent complete
05:08:09.91 95.55 percent complete
05:08:31.77 95.67 percent complete
05:08:53.10 95.78 percent complete
05:09:14.68 95.89 percent complete
05:09:36.63 96.00 percent complete
05:09:58.43 96.11 percent complete
05:10:20.21 96.23 percent complete
05:10:42.03 96.34 percent complete
05:11:03.69 96.45 percent complete
05:11:25.19 96.56 percent complete
05:11:46.40 96.68 pe

05:15:02.54 97.69 percent complete
05:15:23.57 97.80 percent complete
05:15:45.51 97.91 percent complete
05:16:06.80 98.03 percent complete
05:16:28.52 98.14 percent complete
05:16:50.39 98.25 percent complete
05:17:11.52 98.36 percent complete
05:17:33.51 98.48 percent complete
05:17:55.57 98.59 percent complete
05:18:17.29 98.70 percent complete
05:18:38.84 98.81 percent complete
05:19:00.31 98.93 percent complete
05:19:21.87 99.04 percent complete
05:19:43.12 99.15 percent complete
05:20:05.52 99.26 percent complete
05:20:27.41 99.37 percent complete
05:20:48.76 99.49 percent complete
05:21:10.79 99.60 percent complete
05:21:32.23 99.71 percent complete
05:21:53.78 99.82 percent complete
05:22:14.95 99.94 percent complete


In [ ]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# TODO: if your corpus is smaller than 1000, reduce this number. With a corpus that small you might not obtain good results with NMT though :/
# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)


with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(str(row["source_sentence"])+"\n")
    trg_file.write(str(row["target_sentence"])+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(str(row["source_sentence"])+"\n")
    trg_file.write(str(row["target_sentence"])+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# TODO: Doublecheck the format below. There should be no extra quotation marks or weird characters. It should also not be empty.
! head train.*
! head dev.*

==> train.en <==
Our hosts give us a few things to think about .
How can the words at John 13 : 17 prove true in connection with Christian weddings and gatherings ?
Reveal your concerns gradually .
Logically , then , to the extent that Christendom ’ s missionaries advocated democracy and praised the benefits of Western scientific and medical advancement , they came across as agents of colonialism .
18 Paul prayed that his fellow believers in Colossae would be filled with the accurate knowledge of God ’ s will in order for them to walk worthily of Jehovah , please him , and bear fruit in every good work .
Jaime , a taxi driver in Madrid , Spain , has had to live with traffic jams for over 30 years .
Jehovah ’ s Witnesses are helping “ all sorts of men [ to ] be saved and come to an accurate knowledge of truth . ” — 1 Timothy 2 : 4 .
By cooperating fully with them — for example , by sharing in necessary work in the Kingdom Hall .
( Galatians 6 : 7-9 ) Jehovah will put the wicked “ on sli

Installatiom of JOEY NMT

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3224, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 3224 (delta 157), reused 206 (delta 134), pack-reused 2951
Receiving objects: 100% (3224/3224), 8.17 MiB | 18.67 MiB/s, done.
Resolving deltas: 100% (2186/2186), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 8.7 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 392 kB 34.0 MB/s 
     |████████████████████████████████| 246 kB 49.2 MB/s

In [ ]:
! sudo pip3 install subword-nmt

Preprocessing the Data into Subword BPE Tokens
One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization (Sennrich, 2015) .

It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages (Sennrich, 2019) (Martinus, 2019)

Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by (Sennrich, 2019). You do not need to change anything. Simply running the below will be suitable.

In [ ]:

# Do subword NMT
from os import path
os.environ["src"] = source_language 
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language)

! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path


bpe.codes.4000	dev.en	     test.bpe.sw  train.bpe.en	train.sw
dev.bpe.en	dev.sw	     test.en	  train.bpe.sw
dev.bpe.sw	test.bpe.en  test.sw	  train.en


In [ ]:
#### Build vocabulary file using joeynmt

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py ../data/supervised/$src$tgt/train.bpe.$src ../data/supervised/$src$tgt/train.bpe.$tgt --output_path ../data/supervised/$src$tgt/vocab.txt

! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

! echo "BPE Swahili Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt 


BPE Swahili Sentences
Watu wengi hu@@ chang@@ ia kupitia mt@@ and@@ a@@ oni na m@@ fu@@ ko wa pamoja wa S@@ S@@ F kila m@@ r@@ adi hu@@ jit@@ egem@@ e@@ a.
F@@ at@@ en ana@@ hus@@ ika na kur@@ at@@ ibu huduma zinazo@@ tolewa shu@@ l@@ eni mara nyingi nyumbani kwa@@ ke@@ .
W@@ ote wat@@ atu hut@@ engene@@ za ti@@ mu ya wat@@ endaji wa@@ kuu na hufanya shughu@@ li zao katika ta@@ as@@ isi isi@@ yo ya kis@@ er@@ ikali pamoja na kazi zao bin@@ af@@ si@@ .
Mara kwa mara huwa wanat@@ um@@ i@@ ana picha , vid@@ e@@ o@@ , na z@@ ikiwa na ta@@ ar@@ ifa mpya kwenye kurasa za mit@@ and@@ ao ya ki@@ jamii juu ya S@@ S@@ F@@ .
P@@ i@@ a, hus@@ amb@@ aza ma@@ ud@@ hu@@ i yanayo@@ p@@ an@@ ua u@@ elewa kuhusu vita katika nchi ya Y@@ em@@ en@@ , hasa hasa ju@@ kumu la His@@ p@@ ania na nchi nyingine ambazo hus@@ amb@@ aza sil@@ aha zinaz@@ os@@ ababisha mig@@ og@@ oro katika nchi hi@@ yo@@ .Combined BPE Vocab
♕
‐@@
民@@
友@@
̊
̬@@
♡@@
ƫ@@
ĸ@@
➊


In [ ]:
!readlink -f joeynmt

/content/joeynmt


Creating the JoeyNMT Config
JoeyNMT requires a yaml config. 

We used Transformer architecture
We set our dropout to reasonably high: 0.3 (recommended in (Sennrich, 2019))
Things worth playing with:

The batch size (also recommended to change for low-resourced languages)
The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
The decoder options (beam_size, alpha)
Evaluation metrics (BLEU versus Crhf4)

In [ ]:
name = '%s%s' % (source_language, target_language)

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "/content/joeynmt/data/{name}/train.bpe"
    dev:   "/content/joeynmt/data/{name}/dev.bpe"
    test:  "/content/joeynmt/data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/joeynmt/data/{name}/vocab.txt"
    trg_vocab: "/content/joeynmt/data/{name}/vocab.txt"


testing:
    beam_size: 5
    alpha: 1.0

training:
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "/content/joeynmt/models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3


""".format(name=name, data_path="joeynmt", source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

Train the Model
This single line of joeynmt runs the training using the config we made above

In [ ]:
# Train the model
!cd joeynmt; python -m joeynmt train /content/joeynmt/configs/transformer_$src$tgt.yaml

2021-10-25 16:18:28,869 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-25 16:18:29,007 - INFO - joeynmt.data - Loading training data...
2021-10-25 16:18:53,544 - INFO - joeynmt.data - Building vocabulary...
2021-10-25 16:18:53,963 - INFO - joeynmt.data - Loading dev data...
2021-10-25 16:18:54,131 - INFO - joeynmt.data - Loading test data...
2021-10-25 16:18:54,192 - INFO - joeynmt.data - Data loaded.
2021-10-25 16:18:54,192 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-25 16:18:54,647 - INFO - joeynmt.model - Enc-dec model built.
2021-10-25 16:19:00,337 - INFO - joeynmt.training - Total params: 12311808
2021-10-25 16:19:00,340 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-10-25 16:19:00,341 - INFO - joeynmt.helpers - cfg.name                           : ensw_transformer
2021-10-25 16:19:00,341 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-10-25 16:19:00,

A large part of this notebook was adopted from the Masakhane starter notebook.